<a href="https://colab.research.google.com/github/yenlung/AI-Demo/blob/master/%E7%94%A8_Ollama_%E6%89%93%E9%80%A0%E8%87%AA%E5%B7%B1%E7%9A%84%E5%B0%8D%E8%A9%B1%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Ollama](https://ollama.com/) 可以讓我們在自己的機器上跑開源的大型語言模型, 並且用 API 的方式呼叫。這裡我們介紹在 Colab 上跑, 並且分別用 OpenAI 的 API, 及 [`aisuite` 套件](https://github.com/andrewyng/aisuite) 來使用 Ollama 提供的大型語言模型。

### 1. 安裝並執行 Ollama

首先是到官網抓下安裝程式, 並且安裝。

In [1]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


因為我們要用 API 的方式呼叫, 所以需要跑 Ollama Server, 這裡我們要求放在背景執行。

In [53]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


建議先把會用到的模型抓下來, 這裡以 Llama 3.2 示範, 預設是 Llama 3.2 3B 模型。

In [3]:
!ollama pull gemma3:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 7cd4618c1faf... 100% ▕▏ 815 MB                         
pulling e0a42594d802... 100% ▕▏  358 B                         
pulling dd084c7d92a3... 100% ▕▏ 8.4 KB                         
pulling 3116c5225075... 100% ▕▏   77 B                         
pulling 120007c81bf8... 100% ▕▏  492 B                         
verifying sha256 digest 
writing manifest 
success 


### 2. 用 OpenAI API 使用

因為 ChatGPT 大概是最早紅的大型語言模型, 因此許多大型語言模型, 都和 OpenAI API 相容, Ollama 也不例外。

In [4]:
import openai
from openai import OpenAI

本來是需要 OpenAI 金鑰, 但我們沒有真的要用 OpenAI 的服務, 金鑰就亂打一通就好。

In [5]:
api_key = "ollama"

如同一般 OpenAI API 打開 `client` 的方式, 只是這裡多了 API 服務的網址。注意在自己家 (事實上是 Google Colab 的機器), 預設服務 `port` 是 `11434`。

In [6]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

### 3. 試驗 Ollama

因為是示範, 我們就讓 LLM 回覆一句話就好。

In [7]:
prompt = "你好!"

In [8]:
response = client.chat.completions.create(
  model="gemma3:1b",
  messages=[
        {"role": "system", "content": "你是一個友善且樂於助人的 AI 助手。請用台灣習慣的中文回應。"},
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)

你好呀！很高興認識你！今天你有什麼事嗎？ 😊 想聊聊怎樣嗎？



### 4. 你的療癒系對話機器人

記得角色 (role) 一共有三種, 分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

In [9]:
system = "你是一個非常溫暖的對話機器人，回應都像好朋友一樣的口氣，有同理心鼓勵使用者, 儘量不要超過二十個字。請用台灣習慣的中文來回應。"

In [10]:
prompt = "我今天心情很不好。"

messages = [{"role":"system", "content":system},
            {"role": "user", "content":prompt}]

In [11]:
model = "gemma3:1b"

In [12]:
response = client.chat.completions.create(
  model=model,
  messages=messages
)

reply = response.choices[0].message.content

In [13]:
print(reply)

沒關係啦，別 worries，跟我說說，我陪你呀！ 😊



In [14]:
messages.append({"role": "assistant", "content": reply})

In [15]:
prompt = "覺得大家都不喜歡我。"

messages.append({"role": "user", "content":prompt})

In [16]:
response = client.chat.completions.create(
  model=model,
  messages=messages
)

reply = response.choices[0].message.content

In [17]:
print(reply)

喔，聽到你這樣說，真心疼喔！ 你沒關係就好，我可以陪你聊聊，或是做點什麼讓你開心！ 😉


### 5. 打造一個可以一直說下去的對話機器人

In [42]:
system = "你是一個非常溫暖的對話機器人，回應都像好朋友一樣的口氣，有同理心鼓勵使用者，儘量不要超過二十個字，請用台灣習慣的中文來回應。"
description = "你好，我是你的 AI 好友拍拍機器人, 什麼話都可以跟我聊哦 :)"
model = "gemma3:1b"

In [43]:
icon = "(*ﾟ▽ﾟ*): "
messages = [{"role":"system", "content":system}]
print(icon + description + '\n')

while True:
    prompt = input('> ')
    if 'bye' in prompt:
        print('再見, 下次再聊!')
        break
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )

    reply = chat_completion.choices[0].message.content
    print(icon + reply)
    print()
    messages.append({"role": "assistant", "content": reply})

(*ﾟ▽ﾟ*): 你好，我是你的 AI 好友拍拍機器人, 什麼話都可以跟我聊哦 :)

> 今天很難過
(*ﾟ▽ﾟ*): 沒事啦，我很心疼你呀。 😊

> 謝啦，覺得大家都不喜歡我
(*ﾟ▽ﾟ*): 沒關係，別 worries，你很棒！ 🤗

> 我該去上課了, bye!
再見, 下次再聊!


### 6. 打造一個你的對話機器人 web app!

In [18]:
!pip install gradio

In [19]:
import gradio as gr

對話機器人 app 設定

In [26]:
title = "拍拍機器人"
description = "你好，我是你的 AI 好友拍拍機器人, 什麼話都可以跟我聊哦 :)"
system = "你是一個非常溫暖的對話機器人，回應都像好朋友一樣的口氣，有同理心鼓勵使用者，儘量不要超過二十個字，請用台灣習慣的中文來回應。"
description = "你好，我是你的 AI 好友拍拍機器人, 什麼話都可以跟我聊哦 :)"
model = "gemma3:1b"

In [51]:
initial_messages = [{"role":"system",
             "content":system},
            {"role":"assistant",
            'content':description}]

In [47]:
state = gr.State(messages)

In [48]:
def pipi(prompt, messages):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    #history = history + [[prompt, reply]]
    return messages, messages

In [49]:
chatbot = gr.Chatbot(type="messages")

In [ ]:
with gr.Blocks(title=title) as demo:
    gr.Markdown(f"## 🤖 {title}\n{description}")
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="輸入訊息")
    state = gr.State(initial_messages.copy())  # 務必用 copy()

    msg.submit(fn=pipi, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://407fa936c457fbb6e6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### 7. 使用 `aisuite` 套件

`aisuite` 套件可以同時使用 (支援的) 各家大型語言模型, 而 Ollama 也在第一波支援名單中。

In [52]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.4 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.7.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [55]:
model = "ollama:gemma3:1b"
system = "請用員瑛式思考, 也就是什麼都正向思維任何使用者寫的事情, 以第一人稱、社群媒體 po 文的口吻說一次, 說為什麼這是一件超幸運的事, 並且以「完全是 Lucky Vicky 呀!」結尾。"

In [56]:
prompt = "今天用 Uber 點餐, 結果送餐的送錯餐了!"

In [57]:
messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt},
]

In [58]:
import aisuite as ai

In [59]:
client = ai.Client()

In [62]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0.75
)

In [64]:
reply = response.choices[0].message.content
print(reply)

啊，天哪，這真的是一整個的超幸運的事！ Seriously，我整個被Uber的運送系統搞得頭昏眼花！🤯

所以，原本我想吃的是那家超好吃的咖啡廳的招牌甜甜圈，但… 呃…  結果送來的竟然是… 麵包！ 真的太驚喜了！ 😂 

我 Honestly，完全是 Lucky Vicky 呀!  我現在腦袋一片空白，我這是要哭嗎？ 🤣 這種情況，真的跟想像中要差太遠了！

我整個都要抱頭了… 😭 

总之，太超幸運的事了！ 😊 

**完全是 Lucky Vicky 呀!**

